In [2]:
import pandas as pd

txt = pd.read_csv("sentiments.txt",names=["text","sentiment"],sep=";")
print(txt)
print(txt["sentiment"].value_counts())

txt["sentiment"] = txt["sentiment"].replace({"joy":1,"love":1,"surprise":1,"anger":0,"fear":0,"sadness":0})
print(txt["sentiment"].value_counts())

                                                    text sentiment
0                                i didnt feel humiliated   sadness
1      i can go from feeling so hopeless to so damned...   sadness
2       im grabbing a minute to post i feel greedy wrong     anger
3      i am ever feeling nostalgic about the fireplac...      love
4                                   i am feeling grouchy     anger
...                                                  ...       ...
15995  i just had a very brief time in the beanbag an...   sadness
15996  i am now turning and i feel pathetic that i am...   sadness
15997                     i feel strong and good overall       joy
15998  i feel like this was such a rude comment and i...     anger
15999  i know a lot but i feel so stupid because i ca...   sadness

[16000 rows x 2 columns]
sentiment
joy         5362
sadness     4666
anger       2159
fear        1937
love        1304
surprise     572
Name: count, dtype: int64
sentiment
0    8762
1    7238
Na

C:\Users\aaron\AppData\Local\Temp\ipykernel_21808\2704362827.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  txt["sentiment"] = txt["sentiment"].replace({"joy":1,"love":1,"surprise":1,"anger":0,"fear":0,"sadness":0})


In [3]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(txt["text"],txt["sentiment"],train_size=0.8,random_state=8)

In [4]:
import nltk
nltk.download("stopwords")
nltk.download("wordnet")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aaron\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\aaron\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

lemmatizer = WordNetLemmatizer()
def text_transform(data):
    corpus = []
    for sentence in data:
        item = re.sub('[^a-zA-Z]'," ",sentence) #get rid of anything thats not letters
        item = item.lower()
        item = item.split(" ")
        lema = []
        for word in item:
            if word not in stopwords.words("english"):
                lema.append(lemmatizer.lemmatize(word))
        corpus.append(" ".join(lema))
    return corpus

In [6]:
trans_train = text_transform(X_train)
print(trans_train[:10])

['id get class eight dance hour nine get home ten im lucky eat cant eat class dancing full make feel vile sit around digesting etc ish get bed try sleep getting unnaturally early', 'came across picture diy twiggy candle holder im feeling festive creative', 'feel embarrassed talk time feel small moment like favor deserve given attention', 'feel glad starter allowed u interact today', 'need shower child gift feel like caring', 'think folk feeling miserable acknowledge must actually something make world better place', 'feel badly something make really happy', 'feeling almost defeated', 'feeling brave go somewhere afield like walk woodland around farm beach full day activity', 'suspect might also factor making feel rich']


In [11]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(ngram_range=(1,2))
features = vectorizer.fit_transform(trans_train)
trans_test = text_transform(X_test)
X_test = vectorizer.transform(trans_test)
print(features.shape)
print(features[10])

(12800, 88637)
<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 25 stored elements and shape (1, 88637)>
  Coords	Values
  (0, 24794)	1
  (0, 67483)	1
  (0, 36148)	1
  (0, 54927)	1
  (0, 64139)	1
  (0, 1735)	1
  (0, 23321)	1
  (0, 20825)	1
  (0, 86246)	1
  (0, 63208)	1
  (0, 15747)	1
  (0, 33409)	1
  (0, 20761)	1
  (0, 25816)	1
  (0, 67510)	1
  (0, 36207)	1
  (0, 54928)	1
  (0, 64140)	1
  (0, 1742)	1
  (0, 23333)	1
  (0, 20864)	1
  (0, 86327)	1
  (0, 63214)	1
  (0, 15748)	1
  (0, 33414)	1


In [12]:
from sklearn.ensemble import RandomForestClassifier
rf_classifier = RandomForestClassifier()
rf_classifier.fit(features,y_train)
predictions = rf_classifier.predict(X_test)

In [13]:
from sklearn.metrics import classification_report
err = classification_report(y_test,predictions)
print(err)

              precision    recall  f1-score   support

           0       0.95      0.97      0.96      1743
           1       0.96      0.94      0.95      1457

    accuracy                           0.95      3200
   macro avg       0.95      0.95      0.95      3200
weighted avg       0.95      0.95      0.95      3200



In [16]:
def Analyser(txt):
    txt_trans = text_transform(txt)
    txt_num = vectorizer.transform(txt_trans)
    pred =  rf_classifier.predict(txt_num)
    if pred[0] == 1:
        print("Your text is positive!")
    else:
        print("Your text is negative.")
   


Analyser(["I feel very happy about everything"])
Analyser(["My dog died last week."])
    

Your text is positive!
Your text is negative.
